## Data preparation


### Reading Data


- read_data from dataset

- convert images to gray scale

- convert each image to a vector of 10304 value

- stack 400 vector into a single data matrix



In [2]:
import cv2
import numpy as np

path = "dataset\\s%d\\%d.pgm"

def read_data (): 

    data_matrix = np.empty((0, 10304))  
    
    for folder_counter in range (1, 41):
        for image_counter in range (1, 11):
            image = cv2.imread(path%(folder_counter,image_counter))
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            flattened_image = gray_image.flatten()
            flattened_image = np.array(flattened_image)
            data_matrix = np.append(data_matrix, [flattened_image], axis=0)
            # data_matrix = np.vstack((data_matrix, flattened_image))
    
    print(data_matrix.shape)

    return data_matrix


### Splitting data to training and testing data

In [3]:


def split_data(data_matrix):
    
    test_data = data_matrix[::2] #even  
    training_data = data_matrix[1::2] #odd

    return test_data, training_data

## PCA


In [44]:

def PCA(training_data, alpha):

    mean_vector = np.mean(training_data, axis=0)
   
    centered_training_data = training_data - mean_vector
   
    covariance_matrix = np.cov(np.transpose(centered_training_data), bias= True)
   
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
   
    sorted_indices = np.argsort(eigenvalues)[::-1]
   
    sorted_eigenvalues = eigenvalues[sorted_indices]

    accum = sorted_eigenvalues[0]
  
    sum = np.sum(sorted_eigenvalues)
  
    i = 1
   
    while( accum / sum < alpha):
        accum+=sorted_eigenvalues[i]
        i+=1

    projection_matrix = eigenvectors[:, sorted_indices][:, :i]
  
    reduced_dimensionality_data = np.transpose(projection_matrix).dot(np.transpose(centered_training_data))

    return np.transpose(reduced_dimensionality_data)



arr = np.array([[10,60,90],[20,50,70],[30,50,40],[20,50,60],[10,60,10]])
print(PCA(arr, 0.91))


[[ 35.98943074  -9.8642548 ]
 [ 16.04717787   3.84860823]
 [-14.10610139  12.27120986]
 [  6.05083167   3.82195082]
 [-43.98133889 -10.07751411]]
